# Multi-Processing in Python



## Introduction to Multi-Processing

Multi-processing refers to the ability of a system to support more than one processor at the same time. In Python, the `multiprocessing` module allows you to create processes that can run tasks in parallel. This can significantly improve the performance of your program, especially for CPU-bound tasks that can be divided into independent subtasks.



### Why Use Multi-Processing?

Multi-processing is used to achieve parallelism. This means tasks can be executed simultaneously on multiple cores, making your program run faster by taking advantage of the full computing power available. It's particularly beneficial for tasks that are CPU-intensive and can be divided into smaller, independent tasks.



### Threads vs. Processes: Pros and Cons

The key difference between multi-processing and threading lies in how they operate within the system:

- **Threads** of the same process run in a shared memory space, but each **process** in multi-processing runs in its own memory space.
- Threads are lighter-weight and have less overhead than processes. However, because they share the same memory space, threading can lead to issues like race conditions if not managed correctly.
- Multi-processing avoids these issues by giving each process its own memory space, but it comes with a higher memory and CPU overhead due to the need to duplicate the memory space for each process.

Understanding the difference between threads and processes is crucial when deciding how to implement concurrency in your Python applications. Here's a comparison in the form of a pros and cons table:

| Feature           | Threads                                                                 | Processes                                                               |
|-------------------|-------------------------------------------------------------------------|-------------------------------------------------------------------------|
| **Memory Space**  | Share the same memory space of the parent process.                     | Have their own separate memory space.                                   |
| **Creation**      | Lighter weight and faster to create and destroy.                        | Heavier and slower to start and stop due to the need for more resources.|
| **Communication** | Easier to communicate through shared memory but requires synchronization. | Communication requires IPC (Inter-Process Communication) mechanisms like pipes or queues, which can be more complex but safer. |
| **Overhead**      | Lower because they share resources like memory with the parent process. | Higher because each process needs its own memory and resources.         |
| **Use Case**      | Best for I/O-bound and light CPU-bound tasks.                          | Best for CPU-intensive tasks that can be easily parallelized.           |
| **Safety**        | Prone to issues like race conditions due to shared memory.              | Safer in terms of memory corruption as each process has its own memory. |
| **Efficiency**    | Can be more efficient for quick tasks and I/O operations.               | More efficient for long-running, CPU-intensive tasks.                   |

This table highlights the fundamental differences and considerations when choosing between threads and processes for concurrent execution in Python applications. Threads offer a lightweight, shared-memory model for fast communication and lower overhead but require careful management to avoid common concurrency issues. Processes, on the other hand, provide a more isolated and safer environment, ideal for CPU-heavy operations but at the cost of higher resource consumption and more complex communication needs.




### Example

This code snippet demonstrates the use of Python's `multiprocessing` module to create and run a separate process. The `say_hello` function, designed to print a greeting along with the current timestamp, is executed in parallel to the main program. By starting the process and then joining it, we ensure the main program waits for the process to complete before proceeding.

In [ ]:
from multiprocessing import Process
import time
import datetime

def say_hello():
    time.sleep(1)
    print(f"Hello from process: {datetime.datetime.now()}")


# Create a process
process = Process(target=say_hello)

# Start the process
process.start()

print(f"Hello from main process: {datetime.datetime.now()}")

# Wait for the process to finish
process.join()

print("Process complete.")



- This code snippet uses the `multiprocessing` module to create a new process that runs the `say_hello` function.
- Unlike threads, processes have their own memory space. This means the `say_hello` function runs independently of the main program.
- `process.start()` begins the execution of the process, similar to `thread.start()` for threads.
- The main program continues to run concurrently, demonstrating the parallel execution of processes.
- `process.join()` waits for the process to complete before moving on, ensuring that the main program only finishes after all processes have completed.

This example demonstrates the basics of multi-processing in Python. By using processes instead of threads, Python programs can achieve true parallelism, leveraging multiple CPU cores for concurrent execution of tasks. Multi-processing is particularly useful for CPU-bound tasks that can be easily divided into independent subtasks, allowing each process to execute on its own core.
```

## Real Word Example 

In this example, we will perform some image processing tasks. We will download a
few large images from the internet, convert them to grayscale, blur the images,
and then scale them down to thumbnail size. The downloading is an I/O-bound
activity, and the image manipulations are pretty CPU-intensive activities. We
will run three examples: one without threads or multiprocessing, one using
threads, and the other with multiprocessing, and compare the results in terms of
total time.

👆 Press the play button next to the code below to run it. Nothing will happen but
it will initialize the `download_and_process_image()` function and `IMAGE_URLS`
variable so it can be used by the cells below.


In [ ]:
from PIL import Image, ImageFilter
import requests
from PIL import Image
from io import BytesIO
from PIL import Image, ImageFilter
import os


# URLs for ultra high res images from NASA
IMAGE_URLS = [
    "https://mars.nasa.gov/system/downloadable_items/48800_PIA26242-Ingenuitys_View_of_Sand_Dunes_During_Flight_70.jpg",
    "https://mars.nasa.gov/system/downloadable_items/48811_ZCAM_SOL1052_R0_ZCAM05175_INGENUITY_MDI_E01.png",
    "https://mars.nasa.gov/system/downloadable_items/48730_PIA26205-FigC.png",
    "https://mars.nasa.gov/system/downloadable_items/48642_PIA26202.jpg",
    "https://mars.nasa.gov/system/downloadable_items/48431_PIA25968-FigA.jpg",
    "https://mars.nasa.gov/system/downloadable_items/48217_PIA25830-FigureA.jpg",
    "https://stsci-opo.org/STScI-01HN3AEK5XYSQEC92Y37V1AGYV.tif",
    "https://stsci-opo.org/STScI-01EVVGKMPMHDMK28W6VHFAB2FN.png",
    "https://stsci-opo.org/STScI-01FVYYAGQT81SXBGNJX96BF7RB.tif",
    "https://stsci-opo.org/STScI-01EVT0H99TZK40KGNK18M6EK9E.tif",
    "https://stsci-opo.org/STScI-01EVT0GQ9ZABZZZ90G19AR2SHQ.tif",
    "https://stsci-opo.org/STScI-01EVT0WKMTHPCZ68YEG0Y9WPFG.tif",
    "https://stsci-opo.org/STScI-01EVVBGGRWCXT9WM0EF5XXAXPK.tif",
    "https://stsci-opo.org/STScI-01EVVH0Z1XVJ2T61FQ5XRYV9HK.tif",
    "https://stsci-opo.org/STScI-01EVVH2H1HX8AV52T63T7Z20CA.tif"
    ]

def download_and_process_image(url):
    # Download the image
    print(f"Downloading {url}")
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))

    # Perform CPU-intensive tasks
    print(f"Processing {url}")
    # Convert image to grayscale
    img = img.convert("L")
    # Apply a blur filter
    img = img.filter(ImageFilter.GaussianBlur(10))
    # Scale the image to 128x128 pixels
    img = img.resize((128, 128), resample=Image.LANCZOS)

    # make and output dir for the images
    os.makedirs("processed-images", exist_ok=True)

    # Save the processed image as PNG
    img_name = f"./processed-images/processed_{url.split('/')[-1].split('.')[0]}.png"
    img.save(img_name, 'PNG')
    print(f"Saved {img_name}")


This Python script uses the Pillow library to download high-resolution images
from specified URLs, processes each image by converting it to grayscale,
applying a Gaussian blur, resizing it, and then saving the processed image to
disk. 

<details>

<summary>💡 Click here to learn more how this code works:</summary>

### Imports
```python
from PIL import Image, ImageFilter
import requests
from io import BytesIO
```
- **PIL (Python Imaging Library)**: This is actually Pillow, a fork of PIL, which is used for opening, manipulating, and saving many different image file formats.
- **Image, ImageFilter**: These are imported from PIL for image processing tasks such as applying filters.
- **requests**: This module is used to make HTTP requests in Python. It's used here to download images from the internet.
- **BytesIO**: This is a class from the `io` module that allows to read and write binary data like that of an image as if it were a file.

### Image URLs
```python
IMAGE_URLS = [...]
```
This section defines a list of strings, where each string is a URL pointing to an ultra-high-resolution image. These URLs are the source images that will be downloaded and processed.

### The `download_and_process_image` Function
```python
def download_and_process_image(url):
    ...
```
This function is defined to handle the downloading and processing of a single image, given its URL. It works as follows:

- **Downloading the image**:
  - It prints a message indicating the start of the download process for the given URL.
  - The `requests.get(url)` function fetches the content from the URL.
  - `Image.open(BytesIO(response.content))` converts the binary content fetched from the URL into an image object that can be processed by Pillow.

- **Processing the image**:
  - A message is printed to indicate the processing of the image.
  - `img.convert("L")` converts the image to grayscale (where "L" stands for luminance).
  - `img.filter(ImageFilter.GaussianBlur(5))` applies a Gaussian blur with a radius of 5 pixels to the image. This is a CPU-intensive task, especially for high-resolution images.
  - `img.resize((128, 128), resample=Image.LANCZOS)` resizes the image to 128x128 pixels using the Lanczos resampling filter, which is known for producing high-quality results.

- **Saving the processed image**:
  - The processed image is saved with a new filename that is constructed by appending "processed_" to the original filename extracted from the URL.
  - A message is printed to indicate the saving of the processed image.

This function encapsulates the entire workflow for handling a single image: download, process, and save. The actual execution of this function for each URL in `IMAGE_URLS` would need to be done separately, likely in a loop or through a map function, although this part is not shown in the provided code snippet.

</details>

### Single Thread

In the example below, we will use a traditional single thread to process the
images one by one. You will notice that it takes roughly 10-14 seconds to
complete. 

🔑 Make sure you run the code cell above before running the code cell below.

In [ ]:
import time
start_time = time.time()

for url in IMAGE_URLS:
    download_and_process_image(url)

end_time = time.time()
print(f"Process completed in {end_time - start_time} seconds.")

## Multi-Thread

In the example below, we will distribute our work across multiple threads in an
attempt to reduce the total processing time. Each image will be processed in a
separate thread. Notice how the entire process is much faster and it takes
roughly 7-10 seconds!

🔑 Make sure you run the code cell with the function
`download_and_process_image()` above before running the code cell below. 

In [ ]:
import time
import threading

start_time = time.time()

threads = []
for url in IMAGE_URLS:

    thread = threading.Thread(target=download_and_process_image, args=(url,))
    thread.start()
    threads.append(thread)

# Wait
for thread in threads:
    thread.join()

end_time = time.time()
print(f"Process completed in {end_time - start_time} seconds.")

## Multi-Processing

In the example below, we will distribute our work across multiple processes in an
attempt to reduce the total processing time. Each image will be processed in a
separate process with its own CPU and memory space. Notice how the entire
process is much faster and it takes roughly 6-8 seconds!

🔑 Make sure you run the code cell with the function
`download_and_process_image()` above before running the code cell below. 

In [ ]:
import time
from multiprocessing import Process

start_time = time.time()

processes = []
for url in IMAGE_URLS:

    process = Process(target=download_and_process_image, args=(url,))
    process.start()
    processes.append(process)

# Wait for all the threads to finish
for process in processes:
    process.join()

end_time = time.time()
print(f"Process completed in {end_time - start_time} seconds.")


## Collecting Results from Processes Using a Manager in Python's multiprocessing

When working with Python's `multiprocessing` module for parallel processing, sharing data between processes can be challenging due to each process running in its own memory space. Python provides a `Manager` object in the `multiprocessing` module, which allows us to create shared data structures that can be accessed and modified by all processes.



### Understanding the Manager

A `Manager` is a special object provided by the `multiprocessing` module that controls a server process. This server process holds Python objects and allows other processes to manipulate them using proxies. It's a powerful feature for:

- Sharing data between processes, which is otherwise isolated.
- Implementing synchronization mechanisms to avoid data corruption, ensuring thread-safe operations.

Managers support various types of shared objects like lists, dictionaries, and more, including a `Value` container for single values.



### The Example Explained

This example demonstrates using a `Manager` to share a list and a numeric value between multiple processes to identify prime numbers within a range.


In [ ]:
from multiprocessing import Process, Manager
import multiprocessing

def is_prime(num):
    """Check if a number is prime."""
    if num > 1:
        for i in range(2, int(num / 2) + 1):
            if (num % i) == 0:
                return False
        return True
    else:
        return False

def wrapper(num, prime_numbers, prime_count):
    """Wrapper function to check for prime and store it if it is."""
    if is_prime(num):
        prime_numbers.append(num)
        prime_count.value += 1

manager = Manager()

# Using a Manager to create a shared list and value
prime_numbers = manager.list()
prime_count = manager.Value('i', 0)  # 'i' indicates an integer

processes = []
for i in range(9000, 10000):
    # Each process will call the wrapper function with shared objects
    process = Process(target=wrapper, args=(i, prime_numbers, prime_count))
    processes.append(process)
    process.start()

# Wait for all processes to complete
for process in processes:
    process.join()

# Accessing the shared data
print(f"Prime numbers: {list(prime_numbers)}")
print(f"Total prime numbers found: {prime_count.value}")


### Key Components

- **Manager Object:** Created using `Manager()` context manager, which starts a managed server process that holds the shared objects (`prime_numbers` list and `prime_count` value).
- **Shared Objects:** `prime_numbers` is a shared list to store prime numbers, and `prime_count` is a shared `Value` used to count the number of primes found. The `'i'` in `manager.Value('i', 0)` specifies the data type as integer.
- **Wrapper Function:** Accepts a number, the shared list, and the shared value. It checks if the number is prime and, if so, appends it to the list and increments the count.
- **Process Creation and Management:** Creates a process for each number in the specified range, passing the shared objects as arguments. Starts each process and waits for all to complete with `join`.

### Importance of the Manager

The `Manager` plays a crucial role in this example by enabling:
- **Data Sharing Across Processes:** It allows the creation of shared data structures accessible by multiple processes, crucial for collecting and aggregating results from parallel computations.
- **Synchronization:** Provides mechanisms (like locks) to ensure data consistency and thread-safe operations when shared objects are accessed by multiple processes concurrently.

By using a `Manager`, this example effectively showcases how multiprocessing in Python can be used for parallel computations while safely sharing data and results between processes.